In this notebook we provide the code we intially used.

Here we declared our own CNN model, but due to bad accuracy we decided to go with resnet101 which is the [ALPR](https://https://colab.research.google.com/drive/1N_41ZcDMbm7P9ERnjoVT3PwahGMeHKL-?usp=sharing) notebook.

In [0]:
mydata = ImageFolder(root="working_org", transform=ToTensor())
loader = DataLoader(mydata, batch_size=10, shuffle=True, num_workers=2)

Our model has two convolutional layers each followed by a max_pooling. Covolutional layers are followed by RELU activation layer.

Then there are three fully connected layers before the output layer.

In [0]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 128, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(107648, 2048)
        self.fc3 = nn.Linear(2048, 512)
        self.fc5 = nn.Linear(512, 36)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #print(x.size())
        x = x.view(-1, 107648)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, training=self.training)
        x = self.fc5(x)
        return x

model = Net()
model.cuda()

optimizer = optim.Adam(model.parameters(), lr=1e-4, eps=1e-4)

In [0]:
from IPython.display import clear_output
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()
        output = loss(output, target)
        output.backward()
        optimizer.step()
        if batch_idx %1700 == 0:
            clear_output()
        if batch_idx % 100 == 0:
             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                 epoch, batch_idx * len(data), len(loader.dataset),
                 100. * batch_idx / len(loader), output.data[0]))
        #    print()

In [0]:
for epoch in range(5):
    train(epoch)